In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-21 16:44:43,211 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=1800205648224) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=1800205649952) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=1800205650816) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=1800205649808) was called 1 times
Running: Function "load_module_lookup" (id=1800205651536) was called 1 times
Running: Function "load_ID_lookup" (id=1800205650240) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [17]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [29]:
fa_sel = feat.get_data_faults(fa_floor, modules=[1],PTT = 'PTT011')

#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

In [20]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = None, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level ='None')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'None')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=1800205649664) was called 2 times


In [21]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'None')

In [22]:
X,y = md.gen_feat_var(df,target = 'Blue Tote Loss', features = ['Totes'])
y=y

In [23]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'threshold': -np.inf,
                    'max_features': None}

X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       'random_state': None})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X, y)

Feature Importance Ranking: 
 
    Importance Feature
0         1.0   TOTES

Regression Metrics: 
 
           RF Metrics
MAE         0.039394
MSE         0.002611
RMSE        0.051093
MAPE%      19.470000
ACC%       80.530000
OOB         0.685620
R2_Train    0.956703
R2_Pred     0.806781 

Feature Importance Ranking: 
 
    Importance Feature
0         1.0   TOTES

Regression Metrics: 
 
          XGB Metrics
MAE        0.0456736
MSE       0.00350109
RMSE         0.05917
MAPE%          22.57
ACC%           77.43
OOB              N/A
R2_Train     0.98846
R2_Pred     0.740865 


Number of Selected Features:1 


Number of Selected Features:1 


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.867504
2       0.865183
3       0.853803
4       0.724586
5       0.645329
6       0.808486
7       0.834110
8       0.886238
9       0.697215
10      0.840186
Mean    0.802264
STD     0.078811

Cross Validation Scores RandomForestRegressor(oob_score=True): 
 
        R2 Sc

### Add LM - metrics

In [14]:
sc.lm_coefficients(Linear_mdl, X_test ,y_test)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,Feature,Coefficients,Standard Errors,t values,Probabilities
0,Constant,0.7304,NaN,NaN,NaN
1,TOTES,-0.0111,NaN,NaN,NaN


In [15]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

                            OLS Regression Results                            
Dep. Variable:         Blue Tote Loss   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     1616.
Date:                Mon, 21 Dec 2020   Prob (F-statistic):          5.22e-129
Time:                        16:45:39   Log-Likelihood:                 539.92
No. Observations:                 331   AIC:                            -1076.
Df Residuals:                     329   BIC:                            -1068.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7304      0.012     59.262      0.0

In [58]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

C0801STA002    335
C0803STA112    289
C0703STA102     92
C1401ACH001     90
C0503STA251     87
C0501STA012     76
C0503STA252     73
C0502STA056     62
C0801ACH001     61
C0704STA201     47
C0903ACH113     33
C0903STA112     29
C0901STA012     28
C1003STA251     27
C0503ACH234     24
C1101ACH001     24
C1303STA102     17
C1403ACH113     13
C1103STA102     13
C1101STA012     12
C1403STA112     10
C1003STA252      9
C1401ACH011      8
C1301STA002      8
C0904STA206      7
C0603STA102      7
C0901ACH011      7
C0503RDC101      6
C1203ACH106      6
C0701STA002      6
C0501STA002      5
C1201STA012      5
C0703ACH101      5
C0503ACH233      4
C1201ACH001      4
C0801ACH011      4
C0704STA206      4
C1001STA002      4
C1101STA002      3
C0803STA102      3
C1403STA102      3
C0901RDC017      2
C1401STA012      2
C0701ACH011      2
C1203ACH113      2
C0901STA015      2
C1201ACH011      1
C0501ACH011      1
C0502ACH055      1
C1001STA012      1
C1003ACH235      1
C1003ACH232      1
C0503RDC254 

In [70]:
fa[fa['Asset Code'].isnull()]['Alert'].value_counts()

C10: C1002PTT112_047 external unit fault    457
C10: C1002PTT111_037 external unit fault    430
Name: Alert, dtype: int64